In [54]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import geopandas as geopd
import folium
from folium import plugins
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ramitgoolry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ramitgoolry/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [55]:
df_initial = geopd.read_file('Datasets/Police beats/pd_beats_datasd.geojson')

In [56]:
df_initial.head()

,objectid,beat,div,serv,name,geometry
0,3,935,9,930,NORTH CITY,"MULTIPOLYGON (((-117.20425 32.96202, -117.2043..."
1,7,0,0,0,SAN DIEGO,"MULTIPOLYGON (((-117.22526 32.70267, -117.2252..."
2,9,511,5,510,,"MULTIPOLYGON (((-117.22529 32.70260, -117.2252..."
3,10,722,7,720,NESTOR,"POLYGON ((-117.08714 32.58382, -117.08695 32.5..."
4,11,314,3,310,BIRDLAND,"POLYGON ((-117.15377 32.78024, -117.15425 32.7..."


# Extraction of Data from Dataset

In [140]:
LAT, LON = 1,0

# Getting the coordinates of all the centroids of the shapes in data
points = df_initial.geometry.apply(lambda x : tuple(map(float, re.findall(r'[\d\.-]+' ,str(x.centroid)))))
center = np.mean(points.apply(lambda x : x[LAT])), np.mean(points.apply(lambda x : x[LON]))
center

(32.75070182965627, -117.12253670872774)

In [141]:
# Creating map
m = folium.Map(center, zoom_start = 11)

### Mark each point

In [142]:
for point in points:
    folium.CircleMarker([point[LAT], point[LON]],
                        radius=15,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)

### Creating Heatmap

In [160]:
pointsDf = pd.DataFrame({
    'latitude' : list(points.apply(lambda p : p[LAT])),
    'longitude' : list(points.apply(lambda p : p[LON]))
})

m.add_children(plugins.HeatMap(pointsDf.values, radius = 20))
m